In [16]:
import yfinance as yf
import matplotlib.pyplot as plt

In [24]:
# Define the stock symbol and option parameters
stock_symbol = "AAPL"
strike_price = 50.00
expiration_days = 30  # Days until option expiration
interest_rate = 0.058
volatility = 0.24
steps = 100  # Number of steps in the binomial tree


In [25]:
# Fetch the stock data
stock = yf.Ticker(stock_symbol)
stock_data = stock.history(period="1d", interval="1d")  # Adjust period and interval as needed

In [26]:
stock_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-10-10 00:00:00-04:00,178.100006,179.720001,177.949997,178.389999,43664800,0.0,0.0


In [27]:
# Calculate daily returns
daily_returns = stock_data['Close'].pct_change().dropna()

In [28]:
# Calculate the daily risk-free rate
daily_risk_free_rate = (1 + interest_rate) ** (1 / 252) - 1  # Assuming 252 trading days in a year


In [29]:
# Calculate the parameters for the binomial model
delta_t = expiration_days / steps
u = 1 + (volatility * delta_t) ** 0.5
d = 1 - (volatility * delta_t) ** 0.5

In [30]:
# Initialize option price array and a list to store option prices over time
option_prices = [0] * (steps + 1)
option_prices_over_time = []

# Calculate the option prices at expiration (step backward through the tree)
for i in range(steps + 1):
    stock_price = stock_data['Close'].iloc[-1] * (u ** (steps - i)) * (d ** i)
    option_prices[i] = max(stock_price - strike_price, 0)
    option_prices_over_time.append(option_prices.copy())

# Work backward through the tree to calculate option prices at each step
for step in range(steps, 0, -1):
    for i in range(step):
        option_prices[i] = (
            (1 / (1 + daily_risk_free_rate))
            * (0.5 * option_prices[i] + 0.5 * option_prices[i + 1])
        )
        option_prices_over_time.append(option_prices.copy())
        


In [31]:
# The final option price at the root node is the option's fair value
binomial_option_price = option_prices[0]


In [32]:
print(f"Binomial Option Price: ${binomial_option_price:.2f}")

Binomial Option Price: $159.62
